In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd


file_path1 = '/content/drive/MyDrive/Bestbuy/training_dataset.xlsx'
file_path2 = '/content/drive/MyDrive/Bestbuy/validation_dataset.xlsx'
file_path3 = '/content/drive/MyDrive/Bestbuy/test_dataset.xlsx'

In [ ]:
train_df = pd.read_excel(file_path1)
valid_df = pd.read_excel(file_path2)
test_df = pd.read_excel(file_path3)

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# Download NLTK resources (stopwords and WordNet)
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

import spacy


# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from multiprocessing import Pool, cpu_count

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def remove_small_sentences(text):
    sentences = text.split('.')
    imp_sentences =''
    for s in sentences:
        words = s.split()
        l = len(words)
        if l>2: #removing sentences that are very short
            s_filtered = POS_filter(s)
            imp_sentences += s_filtered+'.'
        else:
            continue
    return imp_sentences

def POS_filter(text):
    doc = nlp(text)

# Extract parts of speech for each word
    #pos_tags = [(token.text, token.pos_) for token in doc]

    filtered_words = [token.text for token in doc if token.pos_ in ['NOUN', 'VERB']]
    return ' '.join(filtered_words)



def customer_says(text):
    #text = substring(text)
    #text = re.sub(r'[^a-z\s]', ' ', text)
    #text = replace_consecutive_xs(text)
    pattern = re.compile(r'customer says (.*?)(agent says|customer says|$)')
    matches = re.findall(pattern, text)


    captured_texts = [match[0] for match in matches if len(match[0].split())>4]

    #captured_texts = [
    #for s in matches:
        #words = s[0].split()
        #l = len(words)
        #if l>4:
            #s_filtered = POS_filter(s[0])
            #captured_texts.append(s_filtered)
        #else:
            #continue


    customer_said = ' '.join(captured_texts)

    #filtered_text = remove_small_sentences(customer_said)
    #filtered_text = POS_filter(filtered_text)

    # Display the result
    return POS_filter(customer_said)


def agent_says(text):
    #text = substring(text)
    #text = re.sub(r'[^a-z\s]', '', text)
    #text = replace_consecutive_xs(text)
    pattern = re.compile(r'agent says (.*?)(agent says|customer says|$)')
    matches = re.findall(pattern, text)


    captured_texts = [match[0] for match in matches if len(match[0].split())>4]
    agent_said = ' '.join(captured_texts)

    # Display the result
    return POS_filter(agent_said)



In [ ]:
print(customer_says(train_df['text'].iloc[209342]))

bought size need return have do amazon bring store


In [ ]:
print(agent_says(train_df['text'].iloc[209342]))

thank calling name have pleasure speak date let put touch post purchase support give answer


In [ ]:
train_df['processed_text'] = train_df['text'].apply(lambda x: customer_says(x)+ ' ' + agent_says(x))
valid_df['processed_text'] = valid_df['text'].apply(lambda x: customer_says(x)+ ' ' + agent_says(x))
test_df['processed_text'] = test_df['text'].apply(lambda x: customer_says(x)+ ' ' + agent_says(x))
#test_df['clean_text'] = test_df['text'].apply(lambda x: agent_says(x))
#test_df[['processed_text','label']].to_excel('test_processed.xlsx')
train_df.head(1)

,text,label,processed_text
0,agent says thanks for contacting geek squad. m...,product availability and stock,had question had question s have have question...


In [ ]:
from nltk.corpus import wordnet

def get_most_common_pos(word):
    synsets = wordnet.synsets(word)

    if synsets:
        pos_counts = {}
        for synset in synsets:
            pos = synset.pos()
            pos_counts[pos] = pos_counts.get(pos, 0) + 1

        most_common_pos = max(pos_counts, key=pos_counts.get)
        return most_common_pos
    else:
        # If no synsets found, default to 'n' (noun)
        return 'x'

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove non-alpha characters and replace with space
    text = re.sub(r'[^a-z\s]', ' ', text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text)

    #pos_tags = pos_tag(words)



    filtered_words = [word for word in words if word not in stop_words]

    # Remove courtesy & non verbal words
    courtesy_words = ['hi', 'hello','hey', 'thank','you','agent','customer','says','calling','uh','mhm','oh','i',
                     'name','phone','number','phonenumber','okay','yeah','um','date','bye','contacting','please','yes','no']
    words = [word for word in filtered_words if word not in courtesy_words]

    # Stemming


    #Lemmatization
    #lemmatizer = WordNetLemmatizer()
    #words = [lemmatizer.lemmatize(word) for word in words]
    #words = [word for word in words if get_most_common_pos(word) in ('n','v')]

    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    cleaned_text = ' '.join(words)

    return cleaned_text

In [ ]:
print(preprocess_text(train_df['processed_text'].iloc[10]))

In [ ]:
valid_df['CleanedText'] = valid_df['processed_text'].apply(preprocess_text)
test_df['CleanedText'] = test_df['processed_text'].apply(preprocess_text)
train_df['CleanedText'] = train_df['processed_text'].apply(preprocess_text)
print(train_df['CleanedText'].head())

In [ ]:
train_df.dropna(subset=['CleanedText'], inplace=True)
train_df.info()

In [ ]:
#vectorizing and removing features that are super rare(<5 docs)
#train_df['tokenized_text'] = train_df['CleanedText'].apply(lambda doc: word_tokenize(doc))

vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(train_df['CleanedText'])
vocabulary = vectorizer.get_feature_names_out()
min_document_frequency = 5
document_frequencies = (X > 0).sum(axis=0)

selected_features = [idx for idx, freq in enumerate(document_frequencies.A[0]) if freq >= min_document_frequency]
filtered_vocabulary = [vocabulary[idx] for idx in selected_features]

In [ ]:
tfidf_vectorizer = TfidfVectorizer(vocabulary=filtered_vocabulary)
X_tfidf = tfidf_vectorizer.fit_transform(train_df['CleanedText'])

In [ ]:
tfidf_train_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

In [ ]:
nb_classifier = MultinomialNB()

# Train the classifier
nb_classifier.fit(tfidf_train_df, train_df['label'])

In [ ]:
X_tfidf_valid = TfidfVectorizer.transform(valid_df['CleanedText'])

# Convert the TF-IDF matrix to a DataFrame for better readability
tfidf_valid = pd.DataFrame(X_tfidf_valid.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

In [ ]:
predictions = nb_classifier.predict(tfidf_valid)
print(predictions[:10])

In [ ]:
actual_labels = valid_df['label'].values
print(actual_labels[:10])

In [ ]:
print("Accuracy score is: ",accuracy_score(actual_labels, predictions)*100)

In [ ]:
f1_score_val = f1_score(actual_labels, predictions, average='weighted')
print(f1_score_val*100)

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
svm_params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
svm_model = GridSearchCV(SVC(), svm_params, cv=3)
svm_model.fit(tfidf_train_df, train_df['label'])
best_model = svm_model.best_estimator_

In [ ]:
svm_pred = svm_model.predict(tfidf_valid)
svm_accuracy = accuracy_score(actual_labels, svm_pred)
print("SVM Accuracy:", svm_accuracy*100)
svm_f1 = f1_score(actual_labels, svm_pred, average='weighted')
print(svm_f1*100)

In [ ]:
#Trying out multiple models
# Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(tfidf_train_df, train_df['label'])

# Random Forest
rf_params = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
rf_model = GridSearchCV(RandomForestClassifier(), rf_params, cv=3)
rf_model.fit(tfidf_train_df, train_df['label'])

# Neural Network
nn_params = {'hidden_layer_sizes': [(50, ), (100, ), (50, 50)], 'max_iter': [100, 200, 300]}
nn_model = GridSearchCV(MLPClassifier(), nn_params, cv=3)
nn_model.fit(tfidf_train_df, train_df['label'])

# Make predictions on the test set

nb_predictions = nb_model.predict(tfidf_valid)
rf_predictions = rf_model.predict(tfidf_valid)
nn_predictions = nn_model.predict(tfidf_valid)

# Evaluate the models
svm_accuracy = accuracy_score(actual_labels, svm_predictions)
nb_accuracy = accuracy_score(actual_labels, nb_predictions)
rf_accuracy = accuracy_score(actual_labels, rf_predictions)
nn_accuracy = accuracy_score(actual_labels, nn_predictions)

print("SVM Accuracy:", svm_accuracy)
print("Naive Bayes Accuracy:", nb_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
print("Neural Network Accuracy:", nn_accuracy)


svm_f1 = f1_score(actual_labels, svm_predictions, average='weighted')
nb_f1 = f1_score(actual_labels, nb_predictions, average='weighted')
rf_f1 = f1_score(actual_labels, rf_predictions, average='weighted')
nn_f1 = f1_score(actual_labels, nn_predictions, average='weighted')

# Print F1 scores
print("SVM F1 Score:", svm_f1)
print("Naive Bayes F1 Score:", nb_f1)
print("Random Forest F1 Score:", rf_f1)
print("Neural Network F1 Score:", nn_f1)